In [186]:
import scipy.io.idl as idl
import numpy as np
from scipy.interpolate import griddata
import idl_plane_to_structured_vtk as idl_to_vtk
reload(idl_to_vtk)

<module 'idl_plane_to_structured_vtk' from 'idl_plane_to_structured_vtk.py'>

# Try to store a single time from Jason's IDL scripts
Store $\vec{B}$, $T$, and $n$.

In [12]:
b_x_measurement_points_timestamp = '115110949362'
b_y_measurement_points_timestamp = '115110949522'
b_z_measurement_points_timestamp = '115110938869'
triple_measurement_points_timestamp = '115112535917'
b_x_measurements_timestamp = '115112539367'
b_y_measurements_timestamp = '115112539384'
b_z_measurements_timestamp = '115112539331'
n_measurements_timestamp = '115112539443'
t_measurements_timestamp = '115112539427'

In [13]:
data_path = '../../cataloging_physics_quantities/output/intermediate/'
data_ending = '.sav'
b_x_measurement_points_file = data_path + b_x_measurement_points_timestamp + data_ending
b_y_measurement_points_file = data_path + b_y_measurement_points_timestamp + data_ending
b_z_measurement_points_file = data_path + b_z_measurement_points_timestamp + data_ending
triple_measurement_points_file = data_path + triple_measurement_points_timestamp + data_ending 
b_x_measurements_file = data_path + b_x_measurements_timestamp + data_ending
b_y_measurements_file = data_path + b_y_measurements_timestamp + data_ending
b_z_measurements_file = data_path + b_z_measurements_timestamp + data_ending
n_measurements_file = data_path + n_measurements_timestamp + data_ending
t_measurements_file = data_path + t_measurements_timestamp + data_ending

In [14]:
b_x_measurement_points = idl.readsav(b_x_measurement_points_file)
b_y_measurement_points = idl.readsav(b_y_measurement_points_file)
b_z_measurement_points = idl.readsav(b_z_measurement_points_file)
triple_measurement_points =  idl.readsav(triple_measurement_points_file)
b_x_measurements = idl.readsav(b_x_measurements_file)
b_y_measurements = idl.readsav(b_y_measurements_file)
b_z_measurements = idl.readsav(b_z_measurements_file)
n_measurements = idl.readsav(n_measurements_file)
t_measurements = idl.readsav(t_measurements_file)

In [61]:
n_resampled, x_grid, y_grid = resample_on_structutred_grid(n_measurements, 0)

In [65]:
mesh = prepare_mesh(x_grid, y_grid, 0.25)

In [66]:
n_prepared = prepare_scalar(n_resampled)

In [68]:
write_to_structured_grid('../output/density2.vts', n_prepared, 'n', mesh)

In [178]:
t_measurements['a_out'].shape

(21, 526)

In [184]:
for time_point in xrange(21):
    file_name = '../output/2015-12-03/temperature_%i.vts' % time_point
    (x_min, x_max,
     y_min, y_max) = idl_to_vtk.determine_sample_bounds([t_measurements])
    (t_resampled, 
     x_grid, y_grid) = idl_to_vtk.resample_scalar(t_measurements,
                                                  time_point,
                                                  x_min, x_max, y_min, y_max,
                                                  to_clip=[0, 94, 0, 94])
    mesh = idl_to_vtk.prepare_mesh(x_grid, y_grid, 0.25)
    t_reshaped = idl_to_vtk.reshape_scalar(t_resampled)
    idl_to_vtk.write_to_structured_grid(file_name, t_reshaped, 't', mesh)

In [189]:
for time_point in xrange(21):
    file_name = '../output/2015-12-03/magnetic_field_offset_%i.vts' % time_point
    vector_dicts = [b_x_measurements, b_y_measurements, b_z_measurements]
    (x_min, x_max,
     y_min, y_max) = idl_to_vtk.determine_sample_bounds(vector_dicts)
    (b_resampled, 
     x_grid, y_grid) = idl_to_vtk.resample_vector(vector_dicts[0], 
                                                  vector_dicts[1],
                                                  vector_dicts[2],
                                                  time_point,
                                                  x_min, x_max, y_min, y_max,
                                                  to_clip=[0, 94, 0, 94]) 
    mesh = idl_to_vtk.prepare_mesh(x_grid, y_grid, 0.3)
    b_vector = idl_to_vtk.reshape_vector(*b_resampled)
    idl_to_vtk.write_to_structured_grid(file_name, b_vector, 'B', mesh)